# Pré-processamento dos Dados

Este notebook realiza o tratamento inicial dos dados de **internações e óbitos por causas cardiovasculares no Brasil**, utilizando informações do sistema **TabNet/DATASUS**.

## Problemas típicos nos arquivos originais:

- Cabeçalhos e rodapés não padronizados;
- Linhas com totais anuais que precisam ser removidas;
- Datas em formato de texto, como `"Janeiro/2020"`;
- Presença do símbolo `"-"` para representar ausência de casos.

Essas etapas garantem a padronização necessária para as análises futuras.


In [6]:
# Importa bibliotecas principais e adiciona caminho para os scripts
import os
import pandas as pd
import locale

from scripts.preprocessamento import carregar_csv, tratar_tabela_datasus


## Configuração de Locale

Alguns campos de data usam nomes de meses por extenso em português. Para garantir a correta conversão para o tipo `datetime`, é necessário ajustar o **locale** do sistema.


In [7]:
# Define locale para datas com mês por extenso em português (ex: Janeiro/2020)
try:
    locale.setlocale(locale.LC_TIME, "pt_BR.UTF-8")  # Linux/macOS
except Exception as e:
    try:
        locale.setlocale(locale.LC_TIME, "Portuguese_Brazil.1252")  # Windows
    except Exception as e2:
        pass

## Leitura dos Arquivos CSV

Os arquivos estão divididos em duas categorias:

- **Morbidade:** internações hospitalares
- **Mortalidade:** óbitos registrados

Cada entrada no dicionário indica:

- O caminho para o arquivo;
- Os parâmetros para leitura (`skiprows`, `skipfooter`), que removem cabeçalhos extras e totais não relevantes para a série temporal.

In [8]:
# Dicionário para carregar os dados com os parâmetros corretos
arquivos_csv = {
    # Morbidade
    "internacoes_trombose": {
        "caminho": "../dados_brutos/morbidade/internacoes_trombose_2018-2025.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "internacoes_arritmias_cardiacas": {
        "caminho": "../dados_brutos/morbidade/internacoes_arritmias_cardiacas_2018-2025.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "internacoes_avc": {
        "caminho": "../dados_brutos/morbidade/internacoes_avc_2018-2025.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "internacoes_infarto": {
        "caminho": "../dados_brutos/morbidade/internacoes_infarto_2018-2025.csv",
        "skiprows": 4,
        "skipfooter": 5
    },

    # Mortalidade
    "obitos_avc": {
        "caminho": "../dados_brutos/mortalidade/obitos_avc_2018-2023.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "obitos_infarto": {
        "caminho": "../dados_brutos/mortalidade/obitos_infarto_2018-2023.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "obitos_insuf_cardiaca": {
        "caminho": "../dados_brutos/mortalidade/obitos_insuf_cardiaca_2018-2023.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "obitos_miocardite": {
        "caminho": "../dados_brutos/mortalidade/obitos_miocardite_2018-2023.csv",
        "skiprows": 4,
        "skipfooter": 5
    },
    "obitos_trombose": {
        "caminho": "../dados_brutos/mortalidade/obitos_trombose_2018-2023.csv",
        "skiprows": 4,
        "skipfooter": 5
    }
}

## Importação e Tratamento dos Dados

Cada arquivo é carregado usando a função `carregar_csv`, que trata:

- Encoding e separador;
- Remoção de rodapés e linhas inválidas.

Em seguida, a função `tratar_tabela_datasus` aplica as seguintes transformações:

- Renomeia a coluna de datas;
- Converte meses em português para o tipo `datetime`;
- Remove linhas de totais;
- Converte os valores para `float`;
- Substitui `"-"` por `0.0`, conforme legenda do TabNet.

O resultado final é armazenado no dicionário `tabelas`.


In [9]:
# Carrega e trata todas as tabelas definidas no dicionário
tabelas = {}

for nome, info in arquivos_csv.items():
    df = carregar_csv(
        caminho=info["caminho"],
        skiprows=info["skiprows"],
        skipfooter=info["skipfooter"]
    )
    if df is not None:
        df_tratado = tratar_tabela_datasus(df)
        tabelas[nome] = df_tratado


## Salvamento dos Arquivos Tratados

As tabelas processadas são salvas em diretórios distintos:

- **morbidade/**
- **mortalidade/**

Formato dos arquivos:

- `.csv`
- Encoding: `utf-8-sig`
- Separador: `;`


In [10]:
# Salva os arquivos de mortalidade tratados no diretório apropriado
tabelas_mortalidade = [
    "obitos_avc", 
    "obitos_infarto",
    "obitos_insuf_cardiaca",
    "obitos_miocardite", 
    "obitos_trombose"
]

os.makedirs("../dados_tratados/mortalidade", exist_ok=True)

for nome in tabelas_mortalidade:
    df = tabelas[nome]
    caminho_saida = f"../dados_tratados/mortalidade/{nome}_tratado.csv"
    df.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=";")

In [11]:
# Salva os arquivos de morbidade tratados no diretório apropriado
tabelas_morbidade = [
    "internacoes_trombose",
    "internacoes_arritmias_cardiacas",
    "internacoes_avc",
    "internacoes_infarto"
]

os.makedirs("../dados_tratados/morbidade", exist_ok=True)

for nome in tabelas_morbidade:
    df = tabelas[nome]
    caminho_saida = f"../dados_tratados/morbidade/{nome}_tratado.csv"
    df.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=";")

## Scripts Utilizados

**Arquivo:** `scripts/preprocessamento.py`

### `carregar_csv(...)`

- Lê arquivos CSV com:
  - Encoding `latin1`
  - Separador `;`
  - Linhas ignoradas no início e no fim
  - Remoção de rodapés e linhas inválidas

### `tratar_tabela_datasus(df)`

- Renomeia a coluna de datas
- Converte textos para `datetime`
- Remove linhas de totais
- Converte colunas numéricas para `float`
- Substitui `"-"` por `0.0`
